In [1]:
import torch
import numpy as np
from argparse import ArgumentParser
from double_net.double_net import DoubleNet, train_loop, test_loop_random_start, test_loop
from double_net import datasets as ds
import double_net.plot_utils as pu
import matplotlib.pyplot as plt

In [2]:
train = False
test = True

In [3]:
# import logging
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)

In [4]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [5]:
device

'cuda'

In [6]:
parser = ArgumentParser()
parser.add_argument('--random-seed', type=int, default=0)
parser.add_argument('--num-examples', type=int, default=131072 * 4)
parser.add_argument('--test-num-examples', type=int, default=10000)
parser.add_argument('--test-iter', type=int, default=5)
parser.add_argument('--n-agents', type=int, default=2)
parser.add_argument('--n-items', type=int, default=3)
parser.add_argument('--num-epochs', type=int, default=100)
parser.add_argument('--batch-size', type=int, default=128 * 32)
parser.add_argument('--test-batch-size', type=int, default=10000)
parser.add_argument('--model-lr', type=float, default=1e-3)
parser.add_argument('--misreport-lr', type=float, default=1e-1)
parser.add_argument('--misreport-iter', type=int, default=25)
parser.add_argument('--test-misreport-iter', type=int, default=2000)
parser.add_argument('--rho', type=float, default=1.0)
parser.add_argument('--rho-incr-iter', type=int, default=2)
parser.add_argument('--rho-incr-amount', type=float, default=1.0)
parser.add_argument('--lagr-update-iter', type=int, default=100)
parser.add_argument('--rgt-start', type=int, default=0)
parser.add_argument('--sinkhorn-epsilon', type=float, default=5e-2)
# parser.add_argument('--sinkhorn-rounds', type=int, default=20)
parser.add_argument('--sinkhorn-tol', type=float, default=0.01)

dataset_name = ""
args = parser.parse_args(args=[])
args

Namespace(batch_size=4096, lagr_update_iter=100, misreport_iter=25, misreport_lr=0.1, model_lr=0.001, n_agents=2, n_items=3, num_epochs=100, num_examples=524288, random_seed=0, rgt_start=0, rho=1.0, rho_incr_amount=1.0, rho_incr_iter=2, sinkhorn_epsilon=0.05, sinkhorn_tol=0.01, test_batch_size=10000, test_iter=5, test_misreport_iter=2000, test_num_examples=10000)

In [7]:
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

item_ranges = ds.preset_valuation_range(args.n_agents, args.n_items, dataset_name)
clamp_op = ds.get_clamp_op(item_ranges)

model = DoubleNet(
    args.n_agents, args.n_items, item_ranges, args.sinkhorn_epsilon, args.sinkhorn_tol, marginal_choice='exact_one'
).to(device)

train_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.num_examples, item_ranges).to(device)
train_loader = ds.Dataloader(train_data, batch_size=args.batch_size, shuffle=True)

In [8]:
%%time
if train:
    train_loop(model, train_loader, args, device=device)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.82 µs


In [9]:
import os
dir_name = 'scaled_exact_one_2x3_experiment_data/'
os.makedirs(dir_name, exist_ok=True)

In [10]:
if not train:
    model = DoubleNet.load(dir_name).cuda()
else:
    model.save(dir_name)

In [20]:
%%time
# model.sinkhorn_rounds = 100
# model.sinkhorn_epsilon = 1e-2
if test:
    test_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
    # test_data = torch.load(dir_name + 'test_data').to(device=device)
    cpu_test_data = test_data.clone().to(device='cpu')

    test_loader = ds.Dataloader(test_data, batch_size=args.test_batch_size, shuffle=True)

    result = test_loop(model, test_loader, args, device=device)
    print(result)

1it [00:25, 25.06s/it]


{'payment_mean': 0.19494135677814484, 'regret_mean': 0.004077954683452845, 'regret_max': 0.051655590534210205}
CPU times: user 25 s, sys: 24 ms, total: 25 s
Wall time: 25.1 s


In [21]:
# %%time
if test:
    args.test_num_examples = 1000
    args.test_batch_size = 1000
    args.test_misreport_iter = 1000
    args.misreport_lr = 1e-1
    test_data2 = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
    random_starts = [test_data2]
    for i in range(10):
        random_starts.append(ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device))
    test_loader = ds.Dataloader(test_data2, batch_size=args.test_batch_size, shuffle=True)

    result = test_loop_random_start(model, test_loader, args, random_starts, device=device)
    print(result)

1it [04:26, 266.57s/it]


{'payment_mean': 0.18883170187473297, 'regret_mean': 0.004756520502269268, 'regret_max': 0.05055981129407883}


In [22]:
converged = model.check_convergence(torch.rand(10000,2,3).cuda())

In [23]:
converged[0].max()

tensor(0.0051, device='cuda:0')

In [24]:
converged[1].max()

tensor(9.5367e-07, device='cuda:0')

{'payment_mean': 0.1920904964208603, 'regret_mean': 0.003985989838838577, 'regret_max': 0.04521894454956055}
